# Layers and Modules

- the ResNet-152 architecture, which is wildly popular in computer vision, possesses hundreds of layers. The ResNet architecture won the 2015 ImageNet and COCO computer vision competitions
- From a programming standpoint, a module is represented by a class. Any subclass of it must define a forward propagation method that transforms its input into output and must store any necessary parameters

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2,20)
net(X).shape

/Users/divyamchandel/Documents/Projects/neural-networks-practice/env/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

## Custom Module

<b>Functions for a module</b>
1. Ingest input data as arguments to its forward propagation method
2. Generate an output by having the forward propagation method return a value
3. Calculate the gradient of its output with respect to its input, which can be accessed via its backpropagation method
4. Store and provide access to those parameters necessary for executing the forward propagation computation
5. Initialize model parameters as needed

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X).shape

/Users/divyamchandel/Documents/Projects/neural-networks-practice/env/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

## Executing Forward Prop

- Lets say we want to implement some forward propagation which is custom to our needs. Or using a constant variable which is called as `constant parameters`

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2

        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

/Users/divyamchandel/Documents/Projects/neural-networks-practice/env/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(-0.0483, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.0016, grad_fn=<SumBackward0>)